Download overlay and grab DMA aliases

<div class="alert alert-box alert-warning">
For <b>Kria</b> boards use the `Debug Bridge` IP instead of the JTAG cable.
</div>

In [1]:
from pynq import Overlay, allocate, PL
import numpy as np

PL.reset()

ol = Overlay('dma_subset.bit')
dma = ol.axi_dma
dma_send = ol.axi_dma.sendchannel
dma_recv = ol.axi_dma.recvchannel

In Vivado open **Open Hardware Manger**, then **Open Target** then **Auto Connect**

Allocate input and output buffer, then initialize input buffer. Output buffer is half the size to trigger a Runtime error

In [2]:
data_size = 40

input_buffer = allocate(shape=(data_size,), dtype=np.uint32)
output_buffer_0 = allocate(shape=(data_size//2,), dtype=np.uint32)
output_buffer_1 = allocate(shape=(data_size//2,), dtype=np.uint32)

input_buffer[:] = np.arange(data_size, dtype=np.uint32)

In Vivado Hardware Manager, set the trigger to start capturing when TVALID of *axis_data_fifo_M_AXIS* is 1 or in the rising edge.

Set the number of windows to 2 with a depth of 32 each and the trigger position to 0.

In [ ]:
dma_recv.transfer(output_buffer_0)
dma_send.transfer(input_buffer)
dma_send.wait()
dma_recv.wait()
dma_recv.transfer(output_buffer_1)
dma_recv.wait()

Verify output buffers content

In [ ]:
print(f'{output_buffer_0=}')
print(f'{output_buffer_1=}')

Release the buffers and overlay

In [ ]:
del input_buffer, output_buffer_0, output_buffer_1
ol.free()